# Assignment 2 - Recurrent Neural Network Dynamics 

This coursework is a study of how well a recurrent networo model of primary visual cortex (V1) can represent specific features (we will use the example of orientation) of a brief visual stimulus, in the face of readout noise, depending on its connectivity.

We will begin with the experimental set up:

## Technical Set Up

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Experimental Set Up

### Default Parameters

In [13]:
tau = 20 # characteristic neuron time constant (in ms)

m = 200 # number of regularly spaced orientations

n = 200 # number of neuronal inputs

B = np.eye(m,n) # feedforward input weights

C = np.eye(m,n) # matrix of output weights

sigma = 1 # noise of readout activity 

k = np.pi/4 # encoder constant

alpha = 0.9 # recurrent connection strength constant 1

alpha_dash = 0.9 # recurrent connection strength constant 2

phi = 2 * np.pi * np.linspace(0,1,m) # grid of regularly spaced parameters

dt = 1 # timestep (in ms)

0.0


### Default Functions

In [21]:
# Tuning Function
def V(z):
    return np.exp( (np.cos(z)-1) * k**(-2))


# Input Encoding Function
def h(theta):
    return V(phi-theta)


# Euler Discretised Update Function
def update_r(r_t, t, W, B, h_theta, tau, dt):
    delta_input = (B @ h_theta) / dt if t == 0 else 0
    drdt = -r_t + W @ r_t + delta_input
    return r_t + (dt/tau) * drdt


# Noisy Readout of V1 Activity
def o_tilde(C, r_t, sigma):
    noise = np.random.normal(0,1, size = len(r_t))
    return C @ r_t + sigma * noise 


# Noisy Readout Decoding
def theta_hat(phi, o_tilde):
    x = np.sum(o_tilde * np.sin(phi))
    y = np.sum(o_tilde * np.cos(phi))
    return np.arctan2(y,x)


# Circular Distance 
def d(theta_hat, theta):
    return np.arccos(np.cos(theta_hat-theta))

## Different Models of Recurrence

### **Model 1: no recurrence**
For this model, $n = m$ and $W^{(1)} = 0_m$ (the $m \times m$ matrix full of zeros).

---

### **Model 2: random symmetric connectivity**
For this model, $n = m$ and  
$$
W^{(2)} = \mathcal{R}(\tilde{W} + \tilde{W}^\top, \alpha)
$$  
with $\tilde{W}_{ij} \sim \mathcal{N}(0, 1)$ i.i.d.  
The random weights will be generated once and for all before any simulation of the network dynamics with various $\theta$'s.

---

### **Model 3: symmetric ring structure**
For this model, $n = m$ and  
$$
W^{(3)} = \mathcal{R}(\tilde{W}, \alpha)
$$  
with $\tilde{W}_{ij} = \mathcal{V}(\phi_i - \phi_j)$, where $\{ \phi_i \}$ and $\mathcal{V}(\cdot)$ have been defined previously.

---

### **Model 4: balanced ring structure**
For this model, $n = 2m$, and  
$$
W^{(4)} = 
\begin{pmatrix}
\tilde{W} & -\tilde{W} \\
\tilde{W} & -\tilde{W}
\end{pmatrix}
$$  
with $\tilde{W}_{ij} = \mathcal{R}(W^{(3)}, \alpha')$.  
As $n = 2m$, we can no longer use $B = C = I$ as in Table 1. Instead, we use:

$$
B = \begin{pmatrix} I_m \\ 0_m \end{pmatrix}, \quad 
C = (I_m, 0_m)
$$

where $I_m$ denotes the $m \times m$ identity matrix.


## Questions and Experiments

### Question 1 - Integrating the Dynamics of the 4 different Models